# Практика

Скоринг учащихся (модель для третьего семестра на основе первых двух)

Команда
- Product owner: Антонов Илья
- Scrum-master: Нейман Алексей
- Team: Лебкова Марина, Чвиков Матвей, Махров Матвей, Савин Алексей, Труфманов Михаил

## Загрузка библиотек

In [1]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error, r2_score
from catboost import Pool, CatBoost, CatBoostRegressor, cv

## Предобработка данных

In [3]:
pre_data = pd.read_excel('Успеваемость_01.xlsx')

In [4]:
pre_data.to_csv('/content/new_data.csv', index=False)

In [174]:
df_new = pd.read_csv('new_data.csv')

In [175]:
df_new.head(1)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,Хорошо,Хорошо


In [176]:
df_new = df_new.replace({'I полугодие': 1, 'II полугодие': 2})

#### Удаление данных

Удаление учебных годов (практически) без оценок

In [177]:
years_to_exclude = ['2023 - 2024', '2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']

df_new = df_new[~df_new['Учебный год'].isin(years_to_exclude)].copy()

Удаление групп 22-го года, т.к. для них нет данных о 3 семестре

In [178]:
df_new = df_new[~df_new['Учебная группа'].str.contains('22')].copy()

Удаление предметов без оценки

In [179]:
df_new = df_new.dropna(subset=['Оценка (без пересдач)', 'Оценка (успеваемость)'], how='all')

Заполнение пропусков в промежуточной аттестации при условии, что есть итоговая оценка

In [180]:
df_new['Оценка (без пересдач)'] = df_new.apply(
    lambda row: 2 if pd.isna(row['Оценка (без пересдач)']) and not pd.isna(row['Оценка (успеваемость)']) else row['Оценка (без пересдач)'],
    axis=1
)

#### Добавление столбца с семестром

In [181]:
def calculate_semester(row):
    group_year = int(row['Учебная группа'].split('-')[1])
    start_year = int(row['Учебный год'].split(' - ')[0])
    course = (start_year % 100) - group_year + 1
    semester = course * 2 - 1 if row['Полугодие'] == 1 else course * 2
    return semester

df_new['Семестр'] = df_new.apply(calculate_semester, axis=1)

In [182]:
df_new['Программа'] = df_new['Учебная группа'].str.split('-').str[0]

In [183]:
df_new.head(2)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Иностранный язык,Удовлетворительно,Удовлетворительно,1,БИВТ


Удаление оценок за семестры больше 3

In [184]:
df_new = df_new[df_new['Семестр'] < 4]

In [185]:
disciplines = len(df_new['Дисциплина'].unique())
print(f'Количество уникальных дисциплин: {disciplines}')

Количество уникальных дисциплин: 130


In [186]:
programs = len(df_new['Программа'].unique())
print(f'Количество уникальных программ: {programs}')

Количество уникальных программ: 19


In [187]:
df = df_new
df = df.drop(columns=['Номер ЛД', 'Учебная группа', 'Уровень подготовки', 'Учебный год', 'Полугодие', 'Специальность/направление'])

In [188]:
df.head(1)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ


Так как по ТЗ все оставшиеся null у студента это 2, то заменим их:

#### Кодирование оценок

In [189]:
df.fillna(2, inplace=True)

In [190]:
df.replace({'зачтено': 5, 'Отлично': 5,
            'Хорошо': 4,
            'Удовлетворительно': 3,
            'Неудовлетворительно': 2, 'Неявка': 2, 'не зачтено': 2, 'Не допущен': 2,
            'Неявка по ув.причине': 0
            },
           inplace=True)

In [191]:
df.head(2)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,4,4,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,Иностранный язык,3,3,1,БИВТ


#### Создание столбцов для каждого направления

In [192]:
df_programs = pd.get_dummies(df, columns=['Программа'], prefix='', prefix_sep='')
df_programs = df_programs.drop(columns=['Дисциплина', 'Оценка (без пересдач)', 'Оценка (успеваемость)'])
df_programs = df_programs.astype({col: int for col in df_programs.columns[1:]})
df_programs = df_programs.drop_duplicates()
df_programs = df_programs.drop('Семестр', axis=1)

In [193]:
df_programs.head()

,hash,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,БНМТ,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,67a80fffd8d0294a596eda117d7e393c,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,67a80fffd8d0294a596eda117d7e393c,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99,12c775d99368eb8a655b1cceb39208a5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106,12c775d99368eb8a655b1cceb39208a5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Создание столбцов для каждой дисциплины

Сопоставление итоговых оценок дисциплинам

In [194]:
pivot_df = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (успеваемость)',
                          aggfunc='first'
                          ).reset_index()

pivot_df.columns.name = None
pivot_df.columns = [str(col) for col in pivot_df.columns]
pivot_df.fillna(0, inplace=True)
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns[1:]})
pivot_df = pivot_df[pivot_df['Семестр'].isin([1, 2])].copy()
pivot_df = pivot_df.groupby('hash', as_index=False).sum()
pivot_df = pivot_df.drop(columns=['Семестр'])

In [195]:
pivot_df.head(2)

,hash,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,Введение в языкознание,...,Цифровая экономика,Цифровая экономика и процессное управление предприятием,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0


Сопоставление промежуточных оценок дисциплинам

In [196]:
pivot_df_debts = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (без пересдач)',
                          aggfunc='first'
                          ).reset_index()

pivot_df_debts.columns.name = None
pivot_df_debts.columns = [str(col) for col in pivot_df_debts.columns]
pivot_df_debts.fillna(0, inplace=True)
pivot_df_debts = pivot_df_debts.astype({col: int for col in pivot_df_debts.columns[1:]})
pivot_df_debts = pivot_df_debts[pivot_df_debts['Семестр'] == 3].copy()

In [197]:
pivot_df_debts.head()

,hash,Семестр,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,...,Цифровая экономика,Цифровая экономика и процессное управление предприятием,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,3,2,0,0,0,0,0,0,0
5,00083ac3c8aecc3a1cf66029173e56fa,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,002ce0ab2ee8101b0c19995bdf2b945d,3,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,002ed2297ad196e3b8a7e668f32d125b,3,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
14,0030a813467ddd7c36e9cd12d2b37943,3,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Подсчёт количества двоек в промежуточной аттестации в третьем семестре

In [198]:
df = pivot_df_debts

In [199]:
grades_columns = df.drop(['hash'], axis=1)

In [200]:
df['debts'] = (df == 2).sum(axis=1)
df

,hash,Семестр,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,...,Цифровая экономика и процессное управление предприятием,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение,debts
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,3,2,0,0,0,0,0,0,0,4
5,00083ac3c8aecc3a1cf66029173e56fa,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,002ce0ab2ee8101b0c19995bdf2b945d,3,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11,002ed2297ad196e3b8a7e668f32d125b,3,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,4
14,0030a813467ddd7c36e9cd12d2b37943,3,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10052,ffbcf289933b630e44cc089ff4a873c8,3,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10055,ffc4f5381f1d58a1f6d4c541c71842bc,3,0,0,0,0,5,0,0,0,...,0,0,0,0,0,5,0,0,0,0
10058,ffd3d5961c91b7211451654f570cd7ac,3,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10061,ffe63697b90c27f866b8424c43bc2fdf,3,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
df = pd.merge(pivot_df, df[['hash', 'debts']], on='hash', how='left')

In [202]:
df['Семестр'] = 3 # необходимый столбец для использования двух моделей в одном приложении

In [203]:
df

,hash,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,Введение в языкознание,...,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение,debts,Семестр
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.0,3
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,0,5,...,0,0,0,0,0,0,0,0,0.0,3
2,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,3
3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,5,0,0,0,3,...,0,0,0,0,0,0,0,0,4.0,3
4,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,ffbcf289933b630e44cc089ff4a873c8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3
3400,ffc4f5381f1d58a1f6d4c541c71842bc,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0.0,3
3401,ffd3d5961c91b7211451654f570cd7ac,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,3
3402,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,3


In [204]:
df = df.dropna().copy()
df['debts'] = df['debts'].astype(int).copy()

In [205]:
df

,hash,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,Введение в языкознание,...,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение,debts,Семестр
0,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,3
1,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,0,5,...,0,0,0,0,0,0,0,0,0,3
2,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
3,002ed2297ad196e3b8a7e668f32d125b,0,0,0,0,5,0,0,0,3,...,0,0,0,0,0,0,0,0,4,3
4,0030a813467ddd7c36e9cd12d2b37943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,ffbcf289933b630e44cc089ff4a873c8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3400,ffc4f5381f1d58a1f6d4c541c71842bc,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,3
3401,ffd3d5961c91b7211451654f570cd7ac,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
3402,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


Смена порядка столбцов для удобства

In [206]:
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
print(cols)

['debts', 'Семестр', 'hash', 'Linux', 'Python для извлечения и обработки данных', 'Алгоритмы и структуры данных', 'Базы данных', 'Безопасность жизнедеятельности', 'Бухгалтерский учет и анализ', 'Введение в литературоведение', 'Введение в теорию вероятностей', 'Введение в языкознание', 'Вычислительные машины, сети и системы', 'Геодезия', 'Геология', 'Горное право', 'Горнопромышленная экология', 'Деловой русский язык', 'Дифференциальное исчисление', 'Древние языки и культуры', 'Измерение электрических и неэлектрических величин', 'Инженерная и компьютерная графика', 'Инженерная компьютерная графика', 'Инновационная экономика и технологическое предпринимательство', 'Иностранный язык', 'Инструменты цифрового менеджмента', 'Интенсивный курс русского языка', 'Информатика', 'Искусство продаж', 'История', 'История зарубежной литературы', 'История и культура Великобритании', 'История и культура США', 'История и культура стран испанского языка', 'История и культура стран немецкого языка', 'Истори

In [207]:
df = df[cols]

Сохранение всех дисциплин

In [208]:
lessons = ['Семестр'] + df.columns.to_list()[3:]
print(lessons)

['Семестр', 'Linux', 'Python для извлечения и обработки данных', 'Алгоритмы и структуры данных', 'Базы данных', 'Безопасность жизнедеятельности', 'Бухгалтерский учет и анализ', 'Введение в литературоведение', 'Введение в теорию вероятностей', 'Введение в языкознание', 'Вычислительные машины, сети и системы', 'Геодезия', 'Геология', 'Горное право', 'Горнопромышленная экология', 'Деловой русский язык', 'Дифференциальное исчисление', 'Древние языки и культуры', 'Измерение электрических и неэлектрических величин', 'Инженерная и компьютерная графика', 'Инженерная компьютерная графика', 'Инновационная экономика и технологическое предпринимательство', 'Иностранный язык', 'Инструменты цифрового менеджмента', 'Интенсивный курс русского языка', 'Информатика', 'Искусство продаж', 'История', 'История зарубежной литературы', 'История и культура Великобритании', 'История и культура США', 'История и культура стран испанского языка', 'История и культура стран немецкого языка', 'История и культура стра

In [209]:
df.head(3)

,debts,Семестр,hash,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,...,Цифровая экономика,Цифровая экономика и процессное управление предприятием,Цифровой менеджмент,Человек и общество в социальных науках,Численные методы,Экономика,Экономика производства,Экономика фирмы,Экономическая информатика,Электротехническое и конструкционное материаловедение
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3,002ce0ab2ee8101b0c19995bdf2b945d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Объединение таблицы оценок с таблицей всех направлений

In [210]:
df_programs.head(1)

,hash,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,БНМТ,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [211]:
df_merged = pd.merge(df, df_programs, on=['hash'], how='inner')

In [212]:
df_merged

,debts,Семестр,hash,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,3,000006af6e40c8234a5af27896b7bba5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,3,00083ac3c8aecc3a1cf66029173e56fa,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9980,0,3,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9981,0,3,ffe63697b90c27f866b8424c43bc2fdf,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9982,6,3,ffed4fe2ad8cbff97e0b3d21da23008d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9983,6,3,ffed4fe2ad8cbff97e0b3d21da23008d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Загрузка данных в модель

In [213]:
results = df_merged['debts'].to_numpy()
print(f'Минимальное количество двоек: {results.min()}')
print(f'Максимальное количество двоек: {results.max()}')
print(f'Среднее количество двоек: {results.mean():.3f}')

Минимальное количество двоек: 0
Максимальное количество двоек: 11
Среднее количество двоек: 1.737


In [214]:
df_merged = df_merged.drop('hash', axis=1)

In [215]:
df_merged.head(2)

,debts,Семестр,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Разделение на трейн-тест

In [216]:
X = df_merged.drop('debts', axis=1)
X = X.round().astype(int)
y = df_merged['debts']
y = y.astype(int)

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [218]:
X_train

,Семестр,Linux,Python для извлечения и обработки данных,Алгоритмы и структуры данных,Базы данных,Безопасность жизнедеятельности,Бухгалтерский учет и анализ,Введение в литературоведение,Введение в теорию вероятностей,Введение в языкознание,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
7402,3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2034,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4270,3,0,0,0,0,5,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
8651,3,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9165,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5191,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5390,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
860,3,0,0,0,0,4,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [219]:
y_train

7402    0
2034    2
4270    0
8651    0
9165    3
       ..
5734    4
5191    2
5390    0
860     1
7270    1
Name: debts, Length: 7488, dtype: int64

In [220]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

#### Подбор гиперпараметров

In [221]:
import optuna
from sklearn.model_selection import cross_val_score


def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_seed': 42,
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'loss_function': 'RMSE',
        'eval_metric': 'R2',
    }
    model = CatBoostRegressor(**params, verbose=False)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best trial:')
best_trial = study.best_trial
print(f'  Value: {best_trial.value:.4f}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

[I 2024-07-15 09:12:04,921] A new study created in memory with name: no-name-9f2aeba2-9f41-405b-86f7-4aec55b32491
[I 2024-07-15 09:12:08,735] Trial 0 finished with value: 1.3993014591549884 and parameters: {'iterations': 665, 'depth': 9, 'learning_rate': 0.015455015783167384, 'l2_leaf_reg': 2.761293437714289}. Best is trial 0 with value: 1.3993014591549884.
[W 2024-07-15 09:12:08,869] Trial 1 failed with parameters: {'iterations': 356, 'depth': 4, 'learning_rate': 0.07914094019327156, 'l2_leaf_reg': 5.016097689851747} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-221-95eaddaa130e>", line 17, in objective
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5827, in fit


KeyboardInterrupt: 

In [222]:
model = CatBoostRegressor(custom_metric=['R2'], iterations=1000, learning_rate=0.205, depth=10, l2_leaf_reg=1.16)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

0:	learn: 1.9732883	test: 1.9877173	best: 1.9877173 (0)	total: 7.83ms	remaining: 7.82s
200:	learn: 0.6592260	test: 0.9043822	best: 0.9043822 (200)	total: 1.71s	remaining: 6.79s
400:	learn: 0.5246462	test: 0.7599870	best: 0.7599870 (400)	total: 4.91s	remaining: 7.33s
600:	learn: 0.4825515	test: 0.7056915	best: 0.7056915 (600)	total: 7.16s	remaining: 4.75s
800:	learn: 0.4715506	test: 0.6890494	best: 0.6890494 (800)	total: 8.72s	remaining: 2.17s
999:	learn: 0.4675808	test: 0.6823922	best: 0.6823854 (997)	total: 10.3s	remaining: 0us

bestTest = 0.6823853541
bestIteration = 997

Shrink model to first 998 iterations.


In [223]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2: {r2:.4f}')

MAE: 0.2212
R2: 0.8972


## Класс студента

#### Сохранение всех дисциплин

In [224]:
import json
with open('lessons_third.json', 'w', encoding='utf-8') as file:
    json.dump(lessons, file, ensure_ascii=False)

In [225]:
lessons = lessons # гениально
# в приложении все дисциплины загружаются из json файла
programs = df_programs.columns.to_list()[1:]

class NewStudent:
  def __init__(self):
        self.lessons = lessons
        self.programs = programs
        self.data = {key: 0 for key in list(self.lessons) + list(self.programs)}


  def add_score(self, key, score):
    if key in self.data:
      self.data[key] += score


  def load_data_from_dict(self, data_dict):
      for lesson, score in data_dict.items():
          if lesson in self.data:
              self.data[lesson] += score

  def data_processing(self):
    student_data = np.array(self.prepare_data_for_prediction()).reshape(1, -1)
    student_data = np.nan_to_num(student_data, nan=0, copy=True)
    student_data = student_data.astype(int)
    return student_data

  def prepare_data_for_prediction(self):
    return [self.data[key] for key in self.data]

### Тест модели

In [226]:
student = NewStudent()
student.add_score('БПМ', 1)
student.add_score('Введение в специальность', 5)
student.add_score('Вычислительные машины, сети и системы', 5)
student.add_score('Иностранный язык', 5)
student.add_score('Математика', 4)
student.add_score('Программирование и алгоритмизация', 5)
student.add_score('Инженерная компьютерная графика', 5)
student.add_score('Иностранный язык', 5)
student.add_score('Математика', 3)
student.add_score('Основы дискретной математики', 5)
student.add_score('Объектно-ориентированное программирование', 5)
student.add_score('Персональная эффективность', 5)
student.add_score('Физика', 5)
student.add_score('Физическая культура и спорт', 5)

Предскажем количество двоек в третьем семестре

In [227]:
res = model.predict(student.data_processing()).round().astype(int)
print(f'Количество двоек в третьем семестре: {res[0]}')

Количество двоек в третьем семестре: 1


## Экспорт модели

In [228]:
import pickle

with open('scoring_third_sem.pkl', 'wb') as file:
  pickle.dump(model, file)